In [ ]:
import tensorflow_decision_forests as tfdf  # Importa TensorFlow Decision Forests
import pandas as pd  # Importa Pandas para manejar datos
import numpy as np  # Importa NumPy para operaciones numéricas
import matplotlib.pyplot as plt  # Importa Matplotlib para visualización

# Cargar el conjunto de datos de entrenamiento y prueba desde archivos CSV
train_file_path = "/kaggle/input/house-prices-advanced-regression-techniques/train.csv"
test_file_path = "/kaggle/input/house-prices-advanced-regression-techniques/test.csv"

train_dataset_df = pd.read_csv(train_file_path)  # Carga el conjunto de datos de entrenamiento
test_dataset_df = pd.read_csv(test_file_path)  # Carga el conjunto de datos de prueba

train_dataset_df = train_dataset_df.drop('Id', axis=1)  # Elimina la columna 'Id' del conjunto de datos de entrenamiento

# Define una función para dividir el conjunto de datos en entrenamiento y prueba
def split_dataset(dataset, test_ratio=0.30):
  test_indices = np.random.rand(len(dataset)) < test_ratio  # Elije aleatoriamente ejemplos para el conjunto de prueba
  return dataset[~test_indices], dataset[test_indices]  # Retorna los conjuntos de entrenamiento y prueba

# Aplica la función de división al conjunto de datos de entrenamiento
train_ds_pd, valid_ds_pd = split_dataset(train_dataset_df)
print("{} ejemplos en entrenamiento, {} ejemplos en prueba.".format(len(train_ds_pd), len(valid_ds_pd))  # Muestra la cantidad de ejemplos en entrenamiento y prueba

# Crea datasets de TensorFlow a partir de los dataframes de Pandas
label = 'SalePrice'  # Etiqueta que se va a predecir
train_ds = tfdf.keras.pd_dataframe_to_tf_dataset(train_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)  # Conjunto de entrenamiento
valid_ds = tfdf.keras.pd_dataframe_to_tf_dataset(valid_ds_pd, label=label, task=tfdf.keras.Task.REGRESSION)  # Conjunto de prueba

# Crea un modelo Random Forest para regresión
rf = tfdf.keras.RandomForestModel(task=tfdf.keras.Task.REGRESSION)  # Crea el modelo de Random Forest
rf.compile(metrics=["mse"])  # Compila el modelo y utiliza el MSE como métrica

# Entrena el modelo con los datos de entrenamiento
rf.fit(x=train_ds)  # Entrena el modelo

# Visualiza el modelo
tfdf.model_plotter.plot_model_in_colab(rf, tree_idx=0, max_depth=3)  # Muestra una visualización del modelo

# Calcula e imprime métricas de evaluación en el conjunto de prueba
evaluation = rf.evaluate(x=valid_ds, return_dict=True)  # Evalúa el modelo en el conjunto de prueba
for name, value in evaluation.items():
    print(f"{name}: {value:.4f}")  # Imprime las métricas de evaluación

# Obtiene la importancia de las variables
inspector = rf.make_inspector()  # Inspector para analizar el modelo
variable_importance_metric = "NUM_AS_ROOT"  # Métrica de importancia de variables
variable_importances = inspector.variable_importances()[variable_importance_metric]  # Obtiene la importancia de las variables

# Crea un gráfico de barras de la importancia de las variables
plt.figure(figsize=(12, 4))  # Configura el tamaño de la figura
feature_names = [vi[0].name for vi in variable_importances]  # Nombres de las variables
feature_importances = [vi[1] for vi in variable_importances]  # Importancia de las variables
feature_ranks = range(len(feature_names))  # Rangos de las variables
bar = plt.barh(feature_ranks, feature_importances, label=[str(x) for x in feature_ranks])  # Gráfico de barras
plt.yticks(feature_ranks, feature_names)  # Etiquetas en el eje y
plt.gca().invert_yaxis()  # Invierte el eje y
for importance, patch in zip(feature_importances, bar.patches):
    plt.text(patch.get_x() + patch.get_width(), patch.get_y(), f"{importance:.4f}", va="top")  # Agrega etiquetas en las barras
plt.xlabel(variable_importance_metric)  # Etiqueta del eje x
plt.title("NUM AS ROOT of the class 1 vs the others")  # Título del gráfico
plt.tight_layout()  # Ajusta el diseño
plt.show()  # Muestra el gráfico

# Lee los datos de prueba
test_data = pd.read_csv(test_file_path)
ids = test_data.pop('Id')  # Elimina la columna 'Id' de los datos de prueba
test_ds = tfdf.keras.pd_dataframe_to_tf_dataset(test_data, task=tfdf.keras.Task.REGRESSION)  # Crea un conjunto de datos de prueba
preds = rf.predict(test_ds)  # Realiza predicciones en el conjunto de prueba
output = pd.DataFrame({'Id': ids, 'SalePrice': preds.squeeze()})  # Crea un dataframe con las predicciones

# Guarda los resultados en un archivo CSV
output.to_csv('predicciones.csv', index=False)  # Guarda las predicciones en un archivo CSV